In [1]:
# Datos: https://datosabiertos.bogota.gov.co/dataset/8c2a07f9-5948-41ca-8f7a-447753426ab9/resource/4911565b-4518-4931-93de-d117c10dbdce/download/osb_ocupacion_hosp_ucis.csv

In [2]:
import requests
import pandas as pd

In [3]:
URL = 'https://datosabiertos.bogota.gov.co/dataset/8c2a07f9-5948-41ca-8f7a-447753426ab9/resource/4911565b-4518-4931-93de-d117c10dbdce/download/osb_ocupacion_hosp_ucis.csv'
FILEPATH = './data/camas_bogota_raw.csv'

In [4]:
r = requests.get( URL, stream = True )

with open( FILEPATH, 'wb' ) as f:
    f.write( r.content )

In [5]:
camas = pd.read_csv( './data/camas_bogota_raw.csv', encoding = 'ISO-8859-1', dtype = { 'Camas Ocupadas': str, 'Camas habilitadas': str, 'Camas disponibles': str } )

In [6]:
camas[ 'Camas Ocupadas' ] = camas[ 'Camas Ocupadas' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )
camas[ 'Camas habilitadas' ] = camas[ 'Camas habilitadas' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )
camas[ 'Camas disponibles' ] = camas[ 'Camas disponibles' ].apply( lambda x: str( x ).replace( '.', '' ) ).astype( int )

In [7]:
camas.dtypes

IPS                        object
Servicio                   object
Grupo                      object
Camas Ocupadas              int64
Camas habilitadas           int64
Camas disponibles           int64
Porcentaje de Ocupación    object
Fecha Actualización        object
dtype: object

In [8]:
camas

,IPS,Servicio,Grupo,Camas Ocupadas,Camas habilitadas,Camas disponibles,Porcentaje de Ocupación,Fecha Actualización
0,Públicas,Hospitalización General,Neonatal,107,138,31,78%,13/07/2020
1,Públicas,Hospitalización General,Pediátrico,208,399,191,52%,13/07/2020
2,Públicas,Hospitalización General,Adulto,1070,1314,244,81%,13/07/2020
3,Públicas,Unidad de Cuidado Intermedio,Neonatal,92,126,34,73%,13/07/2020
4,Públicas,Unidad de Cuidado Intermedio,Pediátrico,7,12,5,58%,13/07/2020
5,Públicas,Unidad de Cuidado Intermedio,Adulto,46,48,2,96%,13/07/2020
6,Públicas,Unidad de cuidado intensivo,Neonatal,55,68,13,81%,13/07/2020
7,Públicas,Unidad de cuidado intensivo,Pediátrico,12,29,17,41%,13/07/2020
8,Públicas,Unidad de cuidado intensivo,Adulto,244,280,36,87%,13/07/2020
9,Privadas,Hospitalización General,Neonatal,120,162,42,74%,13/07/2020


In [9]:
camas_grouped = camas.groupby( 'Servicio' ).agg( { 'Camas Ocupadas': 'sum', 'Camas habilitadas': 'sum', 'Camas disponibles': 'sum' } ).reset_index()

In [10]:
camas_grouped[ '% Ocupación' ] = camas_grouped[ 'Camas Ocupadas' ] / camas_grouped[ 'Camas habilitadas' ]

In [11]:
camas_grouped[ 'Servicio' ] = camas_grouped[ 'Servicio' ].replace( { 'Hospitalización General': 'General', 'Unidad de Cuidado Intermedio': 'Intermedio', 'Unidad de cuidado intensivo': 'UCI' } )

In [12]:
camas_grouped.rename( columns = { '% Ocupación': 'Ocupacion' }, inplace = True )

In [13]:
ucis = pd.read_csv( './data/uci_bogota.csv' )

In [14]:
ucis_num = ucis[ 'Camas UCI ocupadas Covid-19' ].values[ -1 ]

In [15]:
ucis_num

1083

In [16]:
listofzeros = [ 0 ] * 3
listofzeros[ list( camas_grouped[ 'Servicio' ].values ).index( 'UCI' ) ] = ucis_num
listofzeros

[0, 0, 1083]

In [17]:
camas_grouped[ 'Camas Ocupadas COVID' ] = pd.Series( listofzeros )

In [18]:
camas_grouped[ 'Camas Ocupadas' ] = camas_grouped[ 'Camas Ocupadas' ] - camas_grouped[ 'Camas Ocupadas COVID' ]

In [19]:
camas_grouped

,Servicio,Camas Ocupadas,Camas habilitadas,Camas disponibles,Ocupacion,Camas Ocupadas COVID
0,General,6739,8658,1919,0.778355,0
1,Intermedio,786,1045,259,0.752153,0
2,UCI,586,2024,355,0.824605,1083


In [20]:
camas_grouped.to_csv( './data/camas_bogota.csv', index = False )